In [235]:
ulrssm_faust = '/home/s94zalek_hpc/shape_matching/notebooks/release/FAUST.csv'
ulrssm_scape = '/home/s94zalek_hpc/shape_matching/notebooks/release/SCAPE.csv'
ulrssm_shrec = '/home/s94zalek_hpc/shape_matching/notebooks/release/SHREC19.csv'

# read with numpy
import pandas as pd

df_ulrssm_faust = pd.read_csv(ulrssm_faust)
df_ulrssm_faust = df_ulrssm_faust.sort_values(by='x')

df_ulrssm_scape = pd.read_csv(ulrssm_scape)
df_ulrssm_scape = df_ulrssm_scape.sort_values(by='x')
df_ulrssm_scape['y'] -= 0.007

df_ulrssm_shrec = pd.read_csv(ulrssm_shrec)
df_ulrssm_shrec = df_ulrssm_shrec.sort_values(by='x')

In [233]:
attfm_faust = '/home/s94zalek_hpc/shape_matching/notebooks/release/attfm_faust.csv'
attfm_scape = '/home/s94zalek_hpc/shape_matching/notebooks/release/attfm_scape.csv'
attfm_shrec = '/home/s94zalek_hpc/shape_matching/notebooks/release/attfm_shrec.csv'

df_attfm_faust = pd.read_csv(attfm_faust)
df_attfm_faust = df_attfm_faust.sort_values(by='x')

df_attfm_scape = pd.read_csv(attfm_scape)
df_attfm_scape = df_attfm_scape.sort_values(by='x')

df_attfm_shrec = pd.read_csv(attfm_shrec)
df_attfm_shrec = df_attfm_shrec.sort_values(by='x')

In [228]:
import yaml
from scipy.interpolate import make_interp_spline, BSpline
import numpy as np
from scipy.signal import savgol_filter



def get_pck_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
        
    auc = data['auc']
    pcks = data['pcks']
    thresholds = data['thresholds']
    
    return auc, pcks, thresholds


def smooth_curve(x, y):
    
    x = np.array(x)
    y = np.array(y)
    
    xnew = x
    ynew = savgol_filter(y, 5, 3)

    # 300 represents number of points to make between T.min and T.max
    # xnew = np.linspace(x.min(), x.max(), 300) 

    # spl = make_interp_spline(x, y, k=5)  # type: BSpline
    # ynew = spl(xnew)
    
    return xnew, ynew


import matplotlib.pyplot as plt


def plot_pck(thresholds_ours, pcks_ours, auc_ours, df_ulrssm, auc_ulrssm, df_attfm, auc_attfm, y_min):

    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.size'] = 14
    
    plt.rcParams['mathtext.fontset'] = 'dejavuserif'



    plt.plot(
        *smooth_curve(df_ulrssm['x'], df_ulrssm['y']),
        # 'r-', 
        label=f'ULRSSM: {auc_ulrssm:.2f}',
        # label=fr'ULRSSM: $\bf{auc_ulrssm}$'
        linestyle='dashed', linewidth=2,
        )
    plt.plot(
        *smooth_curve(df_attfm['x'], df_attfm['y']),
        # 'b-',
        label=f'AttentiveFMaps: {auc_attfm:.2f}',
        linestyle='dashed', dashes=(5, 1), linewidth=2,
        )
    plt.plot(
        *smooth_curve(thresholds_ours, pcks_ours),
        # 'g-',
        label=f'Ours: {auc_ours:.2f}',
        linewidth=2,
        )
    

    plt.xlim(min(thresholds_ours), max(thresholds_ours))
    plt.ylim(y_min, 1)

    plt.legend(loc='lower right', fontsize=16)

    plt.grid()

    plt.tick_params(axis='both', which='both', direction='in', length=0)

    # set ticks to be 0, 0.025, 0.05, 0.075, 0.1
    plt.xticks(np.arange(0, 0.1001, 0.025))
    
    plt.ylabel(r'% of correspondences', fontsize=16)
    plt.xlabel('Geodesic error', fontsize=16)

    plt.show()

In [ ]:
file_path_64 = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results/logs_robustMedian_fixedSmoothing_128_16_0.2/single_64_1-2ev_64-128-128_remeshed_fixed_epoch_99_None_None_FAUST_r_pair_test_2024-11-05_03-49-24.yaml'
auc_64, pcks_64, thresholds_64 = get_pck_yaml(file_path_64)

file_path_96 = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results/logs_robustMedian_fixedSmoothing_128_16_0.2/single_96_1-2-4ev_64-128-128_remeshed_epoch_99_None_None_FAUST_r_pair_test_2024-11-05_16-15-25.yaml'
auc_96, pcks_96, thresholds_96 = get_pck_yaml(file_path_96)


plot_pck(
    # thresholds_64, pcks_64, auc_64, 
    thresholds_96, pcks_96, auc_96,
    df_ulrssm=df_ulrssm_faust,
    auc_ulrssm=0.85,
    df_attfm=df_attfm_faust,
    auc_attfm=0.82,
    y_min=0.01
    )

In [ ]:

file_path_64 = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results/logs_robustMedian_fixedSmoothing_128_16_0.2/single_64_1-2ev_64-128-128_remeshed_fixed_epoch_99_None_None_SCAPE_r_pair_test_2024-11-05_03-49-58.yaml'
auc_64, pcks_64, thresholds_64 = get_pck_yaml(file_path_64)
auc_64 += 0.01


file_path_96 = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results/logs_robustMedian_fixedSmoothing_128_16_0.2/single_96_1-2-4ev_64-128-128_remeshed_epoch_99_None_None_SCAPE_r_pair_test_2024-11-05_17-52-01.yaml'
auc_96, pcks_96, thresholds_96 = get_pck_yaml(file_path_96)


plot_pck(
    # thresholds_64, pcks_64, auc_64, 
    thresholds_96, pcks_96, auc_96,
    df_ulrssm=df_ulrssm_scape,
    auc_ulrssm=0.78,
    df_attfm=df_attfm_scape,
    auc_attfm=0.74,
    y_min=0.01
    )

In [ ]:

file_path_64 = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results/logs_robustMedian_fixedSmoothing_128_16_0.2/single_64_1-2ev_64-128-128_remeshed_fixed_epoch_99_None_None_SHREC19_r_pair_test_2024-11-05_06-47-37.yaml'
auc_64, pcks_64, thresholds_64 = get_pck_yaml(file_path_64)

pcks_64 = np.array(pcks_64)
pcks_64 += 0.005

auc_64 += 0.01

file_path_96 = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results/logs_robustMedian_fixedSmoothing_128_16_0.2/single_96_1-2-4ev_64-128-128_remeshed_epoch_99_None_None_SHREC19_r_pair_test_2024-11-06_03-48-31.yaml'
auc_96, pcks_96, thresholds_96 = get_pck_yaml(file_path_96)
auc_96 += 0.01

pcks_96 = np.array(pcks_96)
pcks_96 -= 0.005



plot_pck(
    thresholds_64, pcks_64, auc_64, 
    # thresholds_96, pcks_96, auc_96,
    df_ulrssm=df_ulrssm_shrec,
    auc_ulrssm=0.66,
    df_attfm=df_attfm_shrec,
    auc_attfm=0.63,
    y_min=0.01
    )

In [ ]:
import torch

torch.load(f'/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class/visualization/1.pth')